# Análisis de características Radiométricas

Se procesan las características radiométricas y se experimentan para obtener el resultado en modelos
**Roberto Araya**


In [108]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold, SelectPercentile, f_regression
#import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [109]:
# parameteres
binwidth = 5
sigma = [1,2,3]
normalize= True
imageTypes = ['Original', 'LoG', 'Square', 'SquareRoot']

sm_radiometrics_file = f'santamaria_data_all__binwidth_{binwidth}_sigma_{sigma}_imtype_{imageTypes}_normalize_{normalize}.csv'
sm_radiometrics_file='santamaria_data_all__binwidth_5_sigma_[1, 2, 3]_normalize_True.csv'
sm_radiometrics = pd.read_csv(sm_radiometrics_file)

In [110]:
sm_radiometrics.head()

,PATIENT_ID,SEXO_MASCULINO,EDAD,FECHA_CIRUGIA,BIOPSIA_QX_PULMONAR,BIOPSIA_FBC-EBUS,BIOPSIA_OTRO_SITIO,RESULTADO_BP,BP_COMPLETA,HISTOLOGIA,...,torax3d_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,torax3d_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,torax3d_wavelet-LLL_glszm_ZoneEntropy,torax3d_wavelet-LLL_glszm_ZonePercentage,torax3d_wavelet-LLL_glszm_ZoneVariance,torax3d_wavelet-LLL_ngtdm_Busyness,torax3d_wavelet-LLL_ngtdm_Coarseness,torax3d_wavelet-LLL_ngtdm_Complexity,torax3d_wavelet-LLL_ngtdm_Contrast,torax3d_wavelet-LLL_ngtdm_Strength
0,sm_001,1,70,NaN,NO,SI,NO,BIOPSIAS TRANSBRONQUIALES-ENDOBRONQUIALES: VAR...,Fecha Informe : 26/03/2018\n\n \n\nINF...,ADENOCARCINOMA,...,3.223680e-10,3.223680e-10,-3.203427e-16,0.000018,0.000000e+00,0.000000,1000000.000000,0.000000,0.000000,0.000000
1,sm_002,0,49,2017-05-19 00:00:00,SI,SI,NO,1.- PUNCION ENDOSONOGRAFICA DE LINFONODOS 4L: ...,Fecha Informe : 17/02/2017\n \n\n\n\nI...,ADENOCARCINOMA,...,1.056094e+00,3.742753e-01,2.663533e+00,0.001273,2.898075e+06,728.494377,0.000856,0.270414,0.065973,0.000842
2,sm_003,0,43,2017-07-05 00:00:00,NO,NO,SI,1.- VENTANA AORTOPULMUNAR : METASTASIS DE ADEN...,Fecha Informe : 10/07/2017\n\n\n\nINFORME ANAT...,ADENOCARCINOMA,...,1.110370e-06,1.110370e-06,-3.203427e-16,0.001054,0.000000e+00,0.000000,1000000.000000,0.000000,0.000000,0.000000
3,sm_004,0,54,2017-05-05 00:00:00,SI,SI,NO,2.- PUNCION EBUS LINFONODO 4L: ABUNDANTE MATER...,Fecha Informe : 04/01/2017\n\nINFORME ANATOMO-...,ADENOCARCINOMA,...,2.114469e-09,2.114469e-09,-3.203427e-16,0.000046,0.000000e+00,0.000000,1000000.000000,0.000000,0.000000,0.000000
4,sm_005,0,44,2016-01-06 00:00:00,SI,NO,SI,2.- LOBECTOMIA SUPERIOR IZQUIERDA (124 GR.): A...,Fecha Informe : 11 DE ENERO DE 2016\n\nINFORME...,ADENOCARCINOMA,...,6.834896e-01,3.172786e-01,3.077820e+00,0.000602,2.531202e+07,728.918817,0.000534,0.140840,0.027048,0.000505


## Procesamiento de datos
- Se eliminan columnas no relevantes.
- Se vectorizan las características compuestas por categorías de *strings* con el método *one-hot-encoding*.
- Se eliminan las columnas que posean alguna fila con valor nulo (INVESIGAR ESTOS CASOS, CASOS SIN EXAMENES TORAX3D).

In [111]:
original_drop_columns = ['PATIENT_ID', 'FECHA_CIRUGIA', 'BIOPSIA_QX_PULMONAR', 'BIOPSIA_FBC-EBUS', 'BIOPSIA_OTRO_SITIO', 'RESULTADO_BP', 'BP_COMPLETA', 'HISTOLOGIA', 'MUTACION_EGFR', 'MUTACION_PDL-1', 'MUTACION_ROS', 'RECIDIVA', 'COMENTARIO', '3D_TORAX_SEG', 'PET_SEG', 'BODY_CT_SEG']
images_columns = ['diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Configuration_Settings', 
                  'diagnostics_Image-original_Dimensionality', 'diagnostics_Mask-original_BoundingBox', 
                  'diagnostics_Versions_PyRadiomics', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Image-original_Hash', 
                  'diagnostics_Image-original_Size', 'diagnostics_Image-original_Spacing', 'diagnostics_Mask-original_Hash', 
                  'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_Spacing', 'diagnostics_Versions_Numpy', 
                  'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Versions_SimpleITK', 
                  'diagnostics_Image-interpolated_Size', 'diagnostics_Image-interpolated_Spacing', 'diagnostics_Mask-interpolated_BoundingBox', 
                  'diagnostics_Mask-interpolated_CenterOfMass', 'diagnostics_Mask-interpolated_CenterOfMassIndex', 'diagnostics_Mask-interpolated_Size',
                  'diagnostics_Mask-interpolated_Spacing']

exam_types = ['body_', 'pet_', 'torax3d_']
images_columns = [exam + s for s in images_columns for exam in exam_types]

# extra columns that are not relevant
extra_columns = ['ALK', 'MUTACION_ALK', 'PDL-1','ROS', 'ADENOPATIAS', 'STAGE', 'TAMAÑO_BP_mm', 'TAMAÑO_CT_mm']

drop_columns = original_drop_columns+images_columns+extra_columns
sm_radiometrics = sm_radiometrics.drop(columns=drop_columns)

# Apply one-hot encoding to selected columns
#sm_radiometrics = pd.get_dummies(sm_radiometrics, columns=['ADENOPATIAS', 'STAGE'])

In [112]:
td_values = [s+'diagnostics_Mask-original_CenterOfMass' for s in exam_types]
print(td_values)

for dim in td_values:
    # Use str.extract to separate the string into three columns with specified suffixes
    extracted_values = sm_radiometrics[dim].str.extract(r'\(([^,]+), ([^,]+), ([^)]+)\)')
    extracted_values.columns = [f'{dim}_x', f'{dim}_y', f'{dim}_z']

    # Convert the columns to numeric (they are currently strings)
    extracted_values = extracted_values.apply(pd.to_numeric)

    # Concatenate the original DataFrame with the extracted values
    sm_radiometrics = pd.concat([sm_radiometrics, extracted_values], axis=1)

# Drop the original column with the (x, y, z) format
sm_radiometrics = sm_radiometrics.drop(td_values, axis=1)


['body_diagnostics_Mask-original_CenterOfMass', 'pet_diagnostics_Mask-original_CenterOfMass', 'torax3d_diagnostics_Mask-original_CenterOfMass']


In [113]:
# Print columns with NaN values
columns_with_nan = sm_radiometrics.columns[sm_radiometrics.isnull().any()].tolist()

# Drop columns with NaN values
sm_radiometrics = sm_radiometrics.drop(columns=columns_with_nan)

## Entrenamiento de modelos y selección de características

Se sigue la metodología realizada por Hector Henriquez en el trabajo [EGFR mutation prediction using F18-FDG PET-CT based radiomics features in non-small cell lung cancer](https://arxiv.org/pdf/2303.08569.pdf) para evaluar el rendimiento de modelos en las caracterterísticas radiométricas.

Algunas configuraciones importantes:
- Se entrena con un modelo RandomForestClassifier.
- Se entrena y evalúa el rendimiento para KFold con $k=3$.
- Se obtienen los resultados en las métricas *accuracy*, *AUC*, *Precision*, *Recall*.

In [114]:
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    
    # Predicted probabilities for class 1 (positive class) for AUC calculation
    probas = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, probas)
    
    # Confusion matrix for true positives and false positives
    cm = confusion_matrix(y, predictions)
    true_positives = cm[1, 1]
    false_positives = cm[0, 1]
    
    # Calculate precision, recall, and avoid division by zero
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + cm[1, 0]) if (true_positives + cm[1, 0]) != 0 else 0
    
    return accuracy, auc, precision, recall

### I. Transformación y filtro de datos
- Se filtran las características de acuerdo a *Low Variance Filter*.
- Se filtran las características con **Selección Univariada de características** con *p-value>0.05*.
- Se estandarizan las variables al intervalo $[0,1]$.

In [115]:
# Separate features and target
X = sm_radiometrics.drop(columns=['EGFR'])
y = sm_radiometrics['EGFR']

# Step 1: Remove features with low variance
variance_threshold = 0.01  # You can adjust this threshold as needed
selector = VarianceThreshold(threshold=variance_threshold)
X_high_variance = selector.fit_transform(X)

# Convert X_high_variance to a DataFrame with the selected features
X_high_variance = pd.DataFrame(X_high_variance, columns=X.columns[selector.get_support()])

# Step 2: Univariate feature selection - SelectPercentile - Filter features with p-value less than 0.05
p_value_threshold = 0.05

percentile_selector = SelectPercentile(f_regression, percentile=100)  # You can adjust the percentile
X_percentile = percentile_selector.fit_transform(X_high_variance, y)
significant_percentile_features = X_high_variance.columns[percentile_selector.pvalues_ < p_value_threshold]
X_filtered_percentile = X_high_variance[significant_percentile_features]

# Step 3: Scale the features to be non-negative and keep the original column names
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X_filtered_percentile), columns=X_filtered_percentile.columns)

print(X.shape)

(35, 24)


### II. Búsqueda de los mejores hiperparámetros del modelo con Grid Search

In [118]:
# Separate features and target
#X = sm_radiometrics.drop(columns=['EGFR'])
#y = sm_radiometrics['EGFR']

# gridsearch
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'criterion': ["gini", "entropy"],
    'max_depth': [None, 10, 20, 30, 40, 50, 100],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 5, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4, 10, 20],
}

rf_model = RandomForestClassifier(random_state=42)
clf = GridSearchCV(rf_model, param_grid)
clf.fit(X, y)

best_estimator = clf.best_estimator_
best_params = clf.best_params_

In [119]:
print(best_params)
print(clf.best_score_)

{'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
0.7714285714285715


### III. Evaluación inicial del modelo

In [117]:
# Number of folds for cross-validation
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=4)

In [107]:
# Initial model performance
initial_results = []
training_results = []

for train_index, test_index in kf.split(X):
    # Train a RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    rf_model.fit(X_train, y_train)
    training_results.append(evaluate_model(rf_model, X_train, y_train))
    initial_results.append(evaluate_model(rf_model, X_test, y_test))


# Calculate mean results for training and testing
training_average_results = np.mean(training_results, axis=0)
initial_average_results = np.mean(initial_results, axis=0)

# Print mean training results
print("Mean Training Results:")
print(f"Accuracy: {training_average_results[0]}")
print(f"AUC: {training_average_results[1]}")
print(f"Precision: {training_average_results[2]}")
print(f"Recall: {training_average_results[3]}")

# Print mean testing results
print("\nMean Testing Results:")
print(f"Accuracy: {initial_average_results[0]}")
print(f"AUC: {initial_average_results[1]}")
print(f"Precision: {initial_average_results[2]}")
print(f"Recall: {initial_average_results[3]}")

TypeError: __init__() got an unexpected keyword argument 'learning_rate'

### IV. Selección de características con *backward selection*
Se filtran las características menos relevantes de acuerdo al proceso de *backward selection* usando el modelo *RandomForestClassifier* con KFold con *k=5*.

In [93]:
## Backward feature selection with k-fold cross-validation
selected_features = list(X.columns)

prev_accuracy = np.inf  # Initialize with a high value
tolerance = 1e-1  # Define a tolerance for stopping criterion

for _ in range(len(selected_features) - 1):
    # Store current performance
    best_accuracy = 0
    best_results = None
    feature_to_remove = None

    # Try removing each feature and evaluate the model using k-fold cross-validation
    for feature in selected_features:
        current_features = [f for f in selected_features if f != feature]
        accuracy_per_fold = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[current_features].iloc[train_index], X[current_features].iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            rf_model = RandomForestClassifier(random_state=42)
            rf_model.fit(X_train, y_train)
            accuracy_per_fold.append(evaluate_model(rf_model, X_test, y_test))

        # Update best feature to remove
        if np.mean(accuracy_per_fold, axis=0)[0] > best_accuracy:
            best_accuracy = np.mean(accuracy_per_fold, axis=0)[0]
            best_results = np.mean(accuracy_per_fold, axis=0)
            feature_to_remove = feature

    # Stop if the change in accuracy is below the tolerance
    if prev_accuracy - best_accuracy < tolerance:
        print("Stopping criterion reached. No significant improvement.")
        break

    prev_accuracy = best_accuracy

    # Remove the least important feature
    selected_features.remove(feature_to_remove)
    print(f"Removed feature {feature_to_remove}, Current results: {best_results}")

# Final selected features
print("Final selected features:", selected_features)

Removed feature body_wavelet-HHL_glrlm_LongRunEmphasis, Current results: [0.71464646 0.78888889 0.75       0.43333333]
Stopping criterion reached. No significant improvement.
Final selected features: ['body_log-sigma-3-mm-3D_glszm_SmallAreaEmphasis', 'body_log-sigma-3-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis', 'body_wavelet-HHH_glszm_SmallAreaEmphasis', 'body_wavelet-HHH_glszm_SmallAreaLowGrayLevelEmphasis', 'body_wavelet-HHL_gldm_LargeDependenceEmphasis', 'body_wavelet-HHL_gldm_LargeDependenceHighGrayLevelEmphasis', 'body_wavelet-HHL_gldm_LargeDependenceLowGrayLevelEmphasis', 'body_wavelet-HHL_glrlm_LongRunHighGrayLevelEmphasis', 'body_wavelet-HHL_glrlm_LongRunLowGrayLevelEmphasis', 'body_wavelet-HHL_glrlm_RunVariance', 'body_wavelet-HHL_glszm_SmallAreaEmphasis', 'body_wavelet-HHL_glszm_SmallAreaHighGrayLevelEmphasis', 'body_wavelet-HLL_glszm_GrayLevelNonUniformityNormalized', 'body_wavelet-HLL_glszm_HighGrayLevelZoneEmphasis', 'body_wavelet-HLL_glszm_LowGrayLevelZoneEmphasis', 'bod

## Version 2: Multidimensional Feature Selection
Se seleccionan las características mas relevantes usando *mfds*.

In [56]:
import mdfs

# Separate features and target
X = sm_radiometrics.drop(columns=['EGFR']).to_numpy()
y = sm_radiometrics['EGFR'].astype(np.intc).to_numpy()

result = mdfs.run(X, y, seed=0, n_contrast=10, dimensions=2, divisions=1, discretizations=1,
        range_=None, pc_xi=0.25, p_adjust_method='fdr_tsbh', level=0.3)

relevant_var = result['relevant_variables']

# Get the names of relevant variables from the original DataFrame
original_column_names = sm_radiometrics.drop(columns=['EGFR']).columns
relevant_column_names = original_column_names[relevant_var]

print(relevant_column_names)

# Create a new DataFrame with the relevant variables and original column names
X = pd.DataFrame(X[:, relevant_var], columns=relevant_column_names)
y = sm_radiometrics['EGFR']

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

Index(['SEXO_MASCULINO', 'EDAD', 'body_diagnostics_Image-interpolated_Maximum',
       'body_diagnostics_Image-interpolated_Mean',
       'body_diagnostics_Image-interpolated_Minimum',
       'body_diagnostics_Image-original_Maximum',
       'body_diagnostics_Image-original_Mean',
       'body_diagnostics_Mask-interpolated_Maximum',
       'body_diagnostics_Mask-interpolated_Mean',
       'body_diagnostics_Mask-interpolated_Minimum',
       ...
       'body_wavelet-LLL_glszm_ZonePercentage',
       'body_wavelet-LLL_glszm_ZoneVariance',
       'body_wavelet-LLL_ngtdm_Busyness', 'body_wavelet-LLL_ngtdm_Coarseness',
       'body_wavelet-LLL_ngtdm_Complexity', 'body_wavelet-LLL_ngtdm_Contrast',
       'body_wavelet-LLL_ngtdm_Strength',
       'body_diagnostics_Mask-original_CenterOfMass_x',
       'body_diagnostics_Mask-original_CenterOfMass_y',
       'body_diagnostics_Mask-original_CenterOfMass_z'],
      dtype='object', length=1304)


In [57]:
# Number of folds for cross-validation
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=4)

# Initial model performance
initial_results = []

for train_index, test_index in kf.split(X):
    # Train a RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    rf_model.fit(X_train, y_train)
    initial_results.append(evaluate_model(rf_model, X_test, y_test))

# Print initial results
initial_average_results = np.mean(initial_results, axis=0)

print("Initial Results:")
print(f"Accuracy: {initial_average_results[0]}")
print(f"AUC: {initial_average_results[1]}")
print(f"Precision: {initial_average_results[2]}")
print(f"Recall: {initial_average_results[3]}")

Initial Results:
Accuracy: 0.6262626262626263
AUC: 0.5341269841269841
Precision: 0.1111111111111111
Recall: 0.16666666666666666


### Preguntas
- Revisar el procesamiento y aplicación del excel (filtros, normalización y otros) para la configuración del extractor - comparar con el paper de Hector.
- Consultar si las imágenes PET de los resultados del paper se realiza la normalización con el PET de liver.
- Se tienen que filtrar las columnas de torax3d porque para algunos pacientes no está aquella información.

### Falta
- Implementar para todos los filtros posibles.
- Normalizar imágenes PET (creo).
- hyperparameter search was performed with gridsearch and the performance metrics were
calculated with 100 repetitions of 5-fold cross-validation.
- Implementar lo anterior para que sea entrenado y validado en Stanford, para luego testear en Santa María.